### Video Processing

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import image_processing as ip
import Lane as Lane

In [2]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [5]:
# Create Class Lane
lanes = Lane.Lanes()
lanes.count = 0
lanes.dif_sum_L = 0
lanes.dif_sum_R = 0
lanes.radius_of_curvature = 100000


def process_image(img):

    img_original = np.copy(img)

    # Warp area
    y_adj = 0

    y_adj = ((1/lanes.radius_of_curvature)**(1/2))*5000
    
        
    
    src = np.float32([[ 0, 680],
                     [ 560, 450],
                     [ 720, 450],
                     [1280, 680]])

    dst = np.float32([[ 250, 720],
                     [ 250, -1000 -y_adj],
                     [1030, -1000 -y_adj],
                     [1030, 720]])

    # warped img
    warped_img, Minv = ip.warped_image(img_original, src, dst)    
    
    
    #---------------------------------#    
    #---------------------------------#
    # Image Processing
    #---------------------------------#
    #---------------------------------#


    #---------------------------------#
    # Sobel
    #---------------------------------#

    gray = cv2.cvtColor(warped_img, cv2.COLOR_RGB2GRAY)
    gray = cv2.blur(gray,(3,3))
    
    kernel = 9
    sobel_thresh = (20,255)

    output1 = ip.sobel_x(gray, kernel, sobel_thresh)

    morph_kernel = np.ones((5,5),np.uint8)
    output1 = cv2.morphologyEx(output1, cv2.MORPH_OPEN, morph_kernel)

    result1 = np.dstack((output1, output1, output1))

    result1_hist = cv2.calcHist([result1],[0], None, [256], [0, 256])
    result1_hist_max = result1_hist[255]
#<#    if result1_hist_max > 32000:
    if result1_hist_max > 320000:
        result1 = np.zeros_like(result1)
    font = cv2.FONT_HERSHEY_PLAIN
    cv2.putText(result1,'thresh: {}'.format(result1_hist_max),(0,20), font, 1,(255,255,255),2,cv2.LINE_AA)        

               
    #---------------------------------#
    # HLS
    #---------------------------------#
        
    h_thresh = ( 16, 28)  # yellow
    
    output5 = ip.hls_select(warped_img, space=0, thresh=h_thresh)
    result5 = np.dstack((output5, output5, output5))


    result5_hist = cv2.calcHist([result5],[0], None, [256], [0, 256])
    result5_hist_max = result5_hist[255]

    if result5_hist_max > 1000000:
        result5 = np.zeros_like(result5)
    font = cv2.FONT_HERSHEY_PLAIN
    cv2.putText(result5,'h_thresh: {}'.format(result5_hist_max),(0,40), font, 1,(255,255,255),2,cv2.LINE_AA)    
    

    result5_left  = result5[:,:639]
    result5_right = result5[:,640:]

    result5_hist_left = cv2.calcHist([result5_left],[0], None, [256], [0, 256])
    result5_hist_right = cv2.calcHist([result5_right],[0], None, [256], [0, 256])
    result5_hist_left_max = result5_hist_left[255]
    result5_hist_right_max = result5_hist_right[255]
    
    if result5_hist_left_max > 20000:
        result5[:,:639] = 0
    if result5_hist_right_max > 20000:
        result5[:,640:] = 0
    cv2.putText(result5,'h_thresh_L: {}'.format(result5_hist_left_max),(0,60), font, 1,(255,255,255),2,cv2.LINE_AA)    
    cv2.putText(result5,'h_thresh_R: {}'.format(result5_hist_right_max),(0,80), font, 1,(255,255,255),2,cv2.LINE_AA)    
    
    
    # HLS S
    s_thresh = ( 75,200)

    output6 = ip.hls_select(warped_img, space=2, thresh=s_thresh)
    result6 = np.dstack((output6, output6, output6))

    result6_hist = cv2.calcHist([result6],[0], None, [256], [0, 256])
    result6_hist_max = result6_hist[255]

    if result6_hist_max > 60000:
        result6 = np.zeros_like(result6)
    font = cv2.FONT_HERSHEY_PLAIN
    cv2.putText(result6,'S_thresh: {}'.format(result6_hist_max),(10,50), font, 3,(255,255,255),2,cv2.LINE_AA)        
    

    
    #---------------------------------#
    # RGB
    #---------------------------------#
        
    color_thresh1 = (170,255)
    color_thresh2 = (240,255)
    
    output4 = ip.rgb_select(warped_img, 0, color_thresh2)
    result4 = np.dstack((output4, output4, output4))

    result4_hist2 = cv2.calcHist([result4],[0], None, [256], [0, 256])
    result4_hist2_max = result4_hist2[255]
    if result4_hist2_max > 15000:   #project = 15000
        result4 = np.zeros_like(result4)
#    else:
#        if lanes.count != 0:
#            if result4_hist2_max <= 15000:
#                result1 = np.zeros_like(result1)
#                result5 = np.zeros_like(result5)
    font = cv2.FONT_HERSHEY_PLAIN
    cv2.putText(result4,'thresh 2: {}'.format(result4_hist2_max),(0,100), font, 1,(255,255,255),2,cv2.LINE_AA)    
    

    if result4_hist2_max < 300:  #1000
        output4 = ip.rgb_select(warped_img, 0, color_thresh1)
        result4 = np.dstack((output4, output4, output4))

        result4_hist1 = cv2.calcHist([result4],[0], None, [256], [0, 256])
        result4_hist1_max = result4_hist1[255]
        if result4_hist1_max > 25000:
            result4 = np.zeros_like(result4)
#<#        else:
#<#            if lanes.count != 0:
#<#                if (3000 < result4_hist1_max) & (result4_hist1_max <= 25000):
#<#                    result1 = np.zeros_like(result1)
#<#                    result5 = np.zeros_like(result5)
            
        font = cv2.FONT_HERSHEY_PLAIN
        cv2.putText(result4,'thresh 2: {}'.format(result4_hist2_max),(0,100), font, 1,(255,255,255),2,cv2.LINE_AA)    
        cv2.putText(result4,'thresh 1: {}'.format(result4_hist1_max),(0,120), font, 1,(255,255,255),2,cv2.LINE_AA)    
        
    
    #---------------------------------#
    # Combination of Processed Images
    #---------------------------------#
        
    result8 = np.array(result1/255 +result4/255 +result5/255 )
    result8[result8 >= 1]=255
    result8[result8 < 1]=0
    
    warped_comb = result8

    
    # 3channel -> 1channel 
    warped_comb_ = np.array(warped_comb[:,:,0]+warped_comb[:,:,1]+warped_comb[:,:,2])
    warped_comb_[warped_comb_<255] = 0
    warped_comb_[warped_comb_>=255] = 1
    warped_comb_.astype(np.uint8)

    # Black mask on warped_comb img
    y_rec1 = 150
    y_rec2 = 350
    
    
    if lanes.radius_of_curvature < 200:
        y_rec1 = 300
        y_rec2 = 450
    if lanes.radius_of_curvature < 150:
        y_rec1 = 400
        y_rec2 = 500
    if lanes.radius_of_curvature < 100:
        y_rec1 = 500
        y_rec2 = 550

    upper_left  = [0,0]
    lower_right = [y_rec1, warped_comb.shape[1]]
    warped_comb_ = ip.black_mask(warped_comb_, upper_left, lower_right)
    
    
    out_img = np.zeros_like(warped_comb) #3channel
    warped_comb2 = np.zeros_like(warped_comb) #3channel

    
    
    #---------------------------------#    
    #---------------------------------#
    # Lane Detecting
    #---------------------------------#
    #---------------------------------#

    
    #---------------------------------#
    # Lane Detecting
    #---------------------------------#

    if (lanes.detected == False) | (lanes.detected_times < 10) :
        out_img, lanes.lane_info = ip.detect_pixels(warped_comb_,9)
        if (len(lanes.lane_info[0]) == 0) | (len(lanes.lane_info[2]) == 0):
            out_img, lanes.lane_info = ip.detect_pixels(warped_comb_,0)

        if lanes.count != 0:
            if (len(lanes.lane_info[0]) == 0) | (len(lanes.lane_info[2]) == 0):
                warped_comb_[lanes.recent_lane_info[0],lanes.recent_lane_info[1]] = 255
                warped_comb_[lanes.recent_lane_info[2],lanes.recent_lane_info[3]] = 255
                out_img, lanes.lane_info = ip.detect_pixels(warped_comb_,5)

        left_fit, right_fit, left_fitx, right_fitx = ip.detect_fit(warped_comb_, lanes.lane_info)   
        check, lanes.dif_sum_L, lanes.dif_sum_R = lanes.check(left_fitx, right_fitx)

        if lanes.count != 0:
            if (check == True):
                pass
            else:
                warped_comb_[lanes.recent_lane_info[0],lanes.recent_lane_info[1]] = 255
                warped_comb_[lanes.recent_lane_info[2],lanes.recent_lane_info[3]] = 255

                out_img, lanes.lane_info = ip.detect_pixels(warped_comb_,5)
                left_fit, right_fit, left_fitx, right_fitx = ip.detect_fit(warped_comb_, lanes.lane_info)   

        lanes.detected = True
        lanes.detected_times += 1
        lanes.recent_lane_info = lanes.lane_info
        lanes.recent_fitx = [left_fitx, right_fitx]


    #---------------------------------#
    # Following Detected Lane
    #---------------------------------#

    else:
        warped_comb_aug = ip.clear_pixels(warped_comb_, lanes.historical_fit[-1])
        warped_comb_aug[0:y_rec2,:] = 0   
        warped_comb_ = np.array(warped_comb_aug) + np.array(warped_comb_)
        warped_comb_[warped_comb_ >= 1] = 1

        res, out_img, lanes.lane_info, left_fit, right_fit, left_fitx, right_fitx = ip.detect_pixels_cont(warped_comb_, lanes.historical_fit[-1])
        if (res == True):
            check, lanes.dif_sum_L, lanes.dif_sum_R = lanes.check(left_fitx, right_fitx)
            if (check == True):
                    lanes.detected = True
                    lanes.detected_times += 1
                    lanes.most_recent_detected_info.append(lanes.lane_info)
                    lanes.recent_lane_info = lanes.lane_info
                    lanes.recent_fitx = [left_fitx, right_fitx]
            else:
                lanes.detected_times = 0
                out_img, lanes.lane_info = ip.detect_pixels(warped_comb_,5)

                if len(lanes.most_recent_detected_info) > 0:
                    for i in range(len(lanes.lane_info)):
                        lanes.lane_info[i] = np.hstack((lanes.lane_info[i],lanes.most_recent_detected_info[-1][i]))

                left_fit, right_fit, left_fitx, right_fitx  = ip.detect_fit(warped_comb_, lanes.lane_info)   
                lanes.detected = True
                lanes.detected_times += 1
                lanes.recent_lane_info = lanes.lane_info
                lanes.recent_fitx = [left_fitx, right_fitx]

        else:
            lanes.detected_times = 0
            out_img, lanes.lane_info = ip.detect_pixels(warped_comb_,5)

            if len(lanes.most_recent_detected_info) > 0:
                for i in range(len(lanes.lane_info)):
                    lanes.lane_info[i] = np.hstack((lanes.lane_info[i],lanes.most_recent_detected_info[-1][i]))

            left_fit, right_fit, left_fitx, right_fitx  = ip.detect_fit(warped_comb_, lanes.lane_info)   
            lanes.detected = True
            lanes.detected_times += 1
            lanes.recent_lane_info = lanes.lane_info
            lanes.recent_fitx = [left_fitx, right_fitx]


    # Depict area
    result_img = ip.depict_area(img, warped_comb_, Minv, left_fit, right_fit)        

    lanes.historical_fit.append([left_fit,right_fit])

    temp_c = lanes.radius_of_curvature

    # Add info radius & offset
    result_img, c, o = ip.add_info(result_img, lanes.lane_info, left_fit, right_fit, lanes.count, lanes.dif_sum_L, lanes.dif_sum_R)


    lanes.radius_of_curvature = int((c*.2+temp_c*.8))
    lanes.line_base_pos = o

    
        
    #---------------------------------#    
    #---------------------------------#
    # Extra Trial
    #---------------------------------#
    #---------------------------------#

        
    # Modify session poly[0]~poly[2]
    mod_left_fit = []
    mod_right_fit = []

    lanes.mod_historical_fit.append([left_fit, right_fit])

    if len(lanes.mod_historical_fit)>100:
        for i in range(3):
            mod_left_fit.append(lanes.adjust_fit(np.array(lanes.historical_fit)[-100:-1,0,i]))
            mod_right_fit.append(lanes.adjust_fit(np.array(lanes.historical_fit)[-100:-1,1,i]))

        lanes.mod_historical_fit[-1] = [mod_left_fit,mod_right_fit]

        mod_result_img = ip.depict_area(img, warped_comb_, Minv, mod_left_fit, mod_right_fit)        

        # Add info radius & offset
        mod_result_img, cc, oo = ip.add_info(mod_result_img, lanes.lane_info, mod_left_fit, mod_right_fit, lanes.count, lanes.dif_sum_L, lanes.dif_sum_R)
        
    else:
        mod_result_img = np.zeros_like(result_img)

    font = cv2.FONT_HERSHEY_PLAIN
    cv2.putText(mod_result_img,'RANSAC',(800,50), font, 4,(255,255,255),2,cv2.LINE_AA)
        
    
    #---------------------------------#    
    #---------------------------------#
    # Output Images
    #---------------------------------#
    #---------------------------------#

        
    # dipict warp area on img
    img_rect = ip.dipict_warp_area(img,src)
    


    result_1 = np.hstack((result_img, warped_img, mod_result_img))
    result_2 = np.hstack((out_img, result1, result4))
    result_3 = np.hstack((warped_comb,result5, result6))
    
    
    result = np.vstack((result_1,result_2,result_3))
    
    lanes.count += 1
    
    return result

In [6]:
project_video_output = 'C:/Users/tyone/Udacity/CarND-Advanced-Lane-Lines/output_images/video502.mp4'  # output file
#clip1 = VideoFileClip("C:/Users/tyone/Udacity/CarND-Advanced-Lane-Lines/project_video.mp4")  # input file
#clip1 = VideoFileClip("C:/Users/tyone/Udacity/CarND-Advanced-Lane-Lines/project_video.mp4").subclip(19,26)  # input file
#clip1 = VideoFileClip("C:/Users/tyone/Udacity/CarND-Advanced-Lane-Lines/challenge_video.mp4")  # input file
#clip1 = VideoFileClip("C:/Users/tyone/Udacity/CarND-Advanced-Lane-Lines/challenge_video.mp4").subclip(2,7)  # input file
clip1 = VideoFileClip("C:/Users/tyone/Udacity/CarND-Advanced-Lane-Lines/harder_challenge_video.mp4")  # input file
#clip1 = VideoFileClip("C:/Users/tyone/Udacity/CarND-Advanced-Lane-Lines/harder_challenge_video.mp4").subclip(25,42)  # input file
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(project_video_output, audio=False)

[MoviePy] >>>> Building video C:/Users/tyone/Udacity/CarND-Advanced-Lane-Lines/output_images/video502.mp4
[MoviePy] Writing video C:/Users/tyone/Udacity/CarND-Advanced-Lane-Lines/output_images/video502.mp4


  8%|██████▎                                                                         | 95/1200 [00:36<07:18,  2.52it/s]

Lane distance error
196.890387175


 18%|██████████████                                                                 | 213/1200 [01:35<08:27,  1.95it/s]

Lane distance error
162.276169618


...

Lane distance error
909.319876185


 92%|███████████████████████████████████████████████████████████████████████▉      | 1106/1200 [09:02<00:43,  2.14it/s]

Lane distance error
999.68019071


100%|█████████████████████████████████████████████████████████████████████████████▉| 1199/1200 [09:49<00:00,  2.01it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: C:/Users/tyone/Udacity/CarND-Advanced-Lane-Lines/output_images/video502.mp4 

Wall time: 9min 52s
